In [ ]:
using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition

In [ ]:
cd(@__DIR__)
isfile("input.txt") ||
  download("https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt",
           "input.txt")

In [ ]:
io = open("input.txt", "r")
text = collect(read(io, String))

* In the above cell:
  1. [read(io, String)](https://docs.julialang.org/en/v1/base/io-network/#Base.read) will read the whole context of the input file as a String.
  1. [collect](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Any}) is a JPL's built-in function which return an `Array` of all items in a collection or iterator.

In [ ]:
alphabet = [unique(text)..., '_']

* The syntax [[]](https://docs.julialang.org/en/v1/manual/functions/#Operators-With-Special-Names-1) is `hcat`.
  * In, JPL [operators are functions](https://docs.julialang.org/en/v1/manual/functions/#Operators-Are-Functions-1). One can apply them using parenthesized argument lists, just as you would any other function.

In [ ]:
stop = onehot('_', alphabet)
text = map(ch -> onehot(ch, alphabet), text)

* `ch -> onehot(ch, alphabet)` is JPL's [anonymous functions](https://docs.julialang.org/en/v0.6.1/manual/functions/#man-anonymous-functions-1).
* [onehot](https://github.com/FluxML/Flux.jl/blob/953280d57f45e1570ef2476b7818216241144187/docs/src/data/onehot.md#one-hot-encoding) is a function implemented in Flux. It converts a word into its index in the dictionary.

In [ ]:
@show N = length(alphabet)

seqlen = 5
nbatch = 2

Xs = collect(partition(batchseq(chunk(text, nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen))

* [chunk](https://github.com/FluxML/Flux.jl/blob/master/src/utils.jl#L25)
* [batchseq](https://github.com/FluxML/Flux.jl/blob/master/src/utils.jl#L79)
* [partition](https://docs.julialang.org/en/v1/base/iterators/#Base.Iterators.partition) is a built-in function in JPL. It iterates over a collection _n_ elements at a time.
* [collect](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Any})
* Here types of `Xs` and `Ys` are `Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1}`.

In [ ]:
m = Chain(
  LSTM(N, 128),
  LSTM(128, 128),
  Dense(128, N),
  softmax)

function loss(xs, ys)
  l = sum(crossentropy.(m.(xs), ys))
  Flux.truncate!(m)
  return l
end

opt = ADAM(params(m), 0.01)
tx, ty = (Xs[1], Ys[1])
evalcb = () -> @show loss(tx, ty)

Flux.train!(loss, zip(Xs, Ys), opt,
            cb = throttle(evalcb, 30))